**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

---


# Introduction

Stack Overflow (stackoverflow.com) is a widely beloved Question and Answer site for technical questions. You'll probably use it yourself as you keep using SQL (or any programming language). 

Their data is publicly available. What cool things do you think it would be useful for?

Here's one idea:
You could set up a service that identifies the Stack Overflow users who have demonstrated expertise with a specific technology by answering related questions about it, so someone could hire those experts for in-depth help.

In this exercise, you'll write the SQL queries that might serve as the foundation for this type of service.

As usual, run the following cell to set up our feedback system before moving on.

In [1]:
# Set up feedack system
from learntools.core import binder
binder.bind(globals())
from learntools.sql.ex6 import *

# import package with helper functions 
import bq_helper

# create a helper object for this dataset
stack_overflow = bq_helper.BigQueryHelper(active_project="bigquery-public-data",
                                              dataset_name="stackoverflow")

Using Kaggle's public dataset BigQuery integration.
Using Kaggle's public dataset BigQuery integration.


# Questions

# 1) Explore the Data

Before writing queries or **JOIN** clauses, you'll want to see what tables are available. 

This may be a good time to practice **tab completion** for when you don't remember command names. If you type `stack_overflow.` and then hit tab, you will see a list of methods for the `stack_overflow` object (don't forget the dot before hitting tab.)

In [3]:
# Your code here

list_of_tables = stack_overflow.list_tables()    # get a list of available tables

print(list_of_tables)
q_1.check()

['badges', 'comments', 'post_history', 'post_links', 'posts_answers', 'posts_moderator_nomination', 'posts_orphaned_tag_wiki', 'posts_privilege_wiki', 'posts_questions', 'posts_tag_wiki', 'posts_tag_wiki_excerpt', 'posts_wiki_placeholder', 'stackoverflow_posts', 'tags', 'users', 'votes']


<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

In [ ]:
# q_1.solution()

# 2) Review Relevant Tables

If you are interested in people who answer questions on a given topic, the `posts_answers` table is a natural place to look. Run the following cell and look at the output.

In [4]:
stack_overflow.head('posts_answers')

,id,body,comment_count,community_owned_date,creation_date,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags
0,4058478,<p>Restart netbeans. I had a similar problem a...,2,None,2010-10-30 11:28:36.983000+00:00,2010-10-30 11:28:36.983000+00:00,None,,None,,409468.0,4057995,2,1,
1,4187003,<p>All URLs within the page are relative to th...,0,None,2010-11-15 17:16:15.927000+00:00,2010-11-15 17:16:15.927000+00:00,None,,None,,439213.0,4184800,2,0,
2,8621524,<p>It sounds like responsiveness to the API ca...,0,None,2011-12-23 23:25:45.430000+00:00,2011-12-23 23:25:45.430000+00:00,None,,None,,1114079.0,8595650,2,1,
3,6553106,<p>Yes. That's the (one of) the points of Hash...,2,None,2011-07-01 20:51:33.830000+00:00,2011-07-01 20:51:33.830000+00:00,None,,None,,13956.0,6553067,2,0,
4,23032552,<p>I think I found the problem. I changed the ...,0,None,2014-04-12 15:43:59.437000+00:00,2014-04-12 15:43:59.437000+00:00,None,,None,user3413108,NaN,22366517,2,3,


It isn't clear yet how to the find users who answered questions on any given topic. But `posts_answers` has a `parent_id` column. If you are familiar with the Stack Overflow site, you might figure out that the `parent_id` is the question each post is answering.

Look at `posts_questions` using the line below.

In [5]:
stack_overflow.head('posts_questions')

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,post_type_id,score,tags,view_count
0,23280293,Jasper report data summary,<p>I am trying to create a report to display a...,23373319.0,1,2,None,2014-04-24 21:33:46.787000+00:00,NaN,2014-05-01 13:37:52.027000+00:00,2014-05-01 13:37:52.027000+00:00,,321731,,1214943,1,0,filter|jasper-reports|summary,266
1,5787776,A worthy developer-friendly alternative to PayPal,"<p>I understand payments are a tricky thing, b...",NaN,8,9,None,2011-04-26 08:28:37.397000+00:00,40.0,2013-03-19 01:57:09.860000+00:00,2013-03-19 01:57:09.860000+00:00,,772853,,50841,1,95,paypal|payment-gateway|payment|credit-card,10505
2,51899406,Why does adding a destructor (even empty) brea...,<p>I am making a class to store a reference to...,NaN,1,0,None,2018-08-17 16:17:18.367000+00:00,NaN,2018-08-17 17:08:03.690000+00:00,2018-08-17 17:08:03.690000+00:00,,241631,,3662349,1,3,c++|c++11|templates|perfect-forwarding,53
3,29219176,"Java Math.pow(x,2.0) vs Math.pow(x,2.0000001) ...",<p>I am trying to compare performance of <code...,29232947.0,1,8,None,2015-03-23 19:55:25.693000+00:00,2.0,2016-07-19 21:48:22.440000+00:00,2016-07-19 21:48:22.440000+00:00,,2753863,,2739693,1,5,java|performance,427
4,45545053,rioBufferWrite function Redis updating position,"<p>When I am reading the code of <a href=""http...",NaN,0,6,None,2017-08-07 10:46:21.007000+00:00,NaN,2017-08-07 10:57:45.507000+00:00,2017-08-07 10:57:45.507000+00:00,,8051589,,7700616,1,1,c|redis,30


Are there any fields that identify what topic or technology each question is about?

If so, how could you find the user ID\'s of users who answered questions about a specific topic?

Think about it, then check the solution by running the code in the next cell.

In [6]:
q_2.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
`posts_questions` has a column called `tags` which lists the topics/technologies each question is about.

`posts_answers` has a column called `parent_id` which identifies the ID of the question each answer is responding to.
`posts_answers` also has an `owner_user_id` column which specifies the ID of the user who answered the question.

You can join these two tables to:
- determine the `tags` for each answer, and then
- select the `owner_user_id` of the answers on the desired tag.

This is exactly what you will do over the next few questions.


# 3 Selecting The Right Questions

A lot of this data is text. 

Here is one last technique you'll learn in this course which you can apply to this text:

A **WHERE** clause can limit your results to rows with certain text using the **LIKE** feature. For example, to select just the third row of the `pets` table, we would write

`SELECT * FROM PETS WHERE NAME LIKE 'Ripley'`

![](https://i.imgur.com/Ef4Puo3.png)

You can also use `%` as a "wildcard" for any number of characters. So you can get the third row with 

`SELECT * FROM PETS WHERE NAME LIKE '%ipl%'`

Try this yourself.
Before finding users who have answered questions, write a query that selects the `id`, `title` and `owner_user_id` from the `posts_questions` table. Restrict the results to rows that contain the word **bigquery** in the `tags` column. Include rows where there is other text in addition to the word `bigquery` (e.g. if a row has a tag `bigquery-sql`, your results should include that too).

In [20]:
# Your code here
questions_query = """
SELECT id, title, owner_user_id                  
FROM `bigquery-public-data.stackoverflow.posts_questions`
WHERE tags LIKE '%bigquery%'
"""

questions_results = stack_overflow.query_to_pandas_safe(questions_query, max_gb_scanned=25) # this query reads a lot of data
print(questions_results.head())
q_3.check()


         id      ...      owner_user_id
0  35498370      ...          3478208.0
1  42300146      ...          2854735.0
2  41394244      ...          3963320.0
3  43974500      ...            66051.0
4  41998344      ...          1047635.0

[5 rows x 3 columns]


<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> The results don't look right. Try again.

In [18]:
# q_3.hint()
# q_3.solution()

# 4 Your First Join
Now that you have a query to select questions on any given topic (in this case, you chose `bigquery`), you can find the answers to those questions with a **JOIN**.  

Write a SQL query that returns the `id`, `body` and `owner_user_id` from the `posts_answers` table for answers to `bigquery` related questions. That is, you should have one row in your results for each answer to a question that has a `bigquery` in the tag.

Here's a reminder of what a **JOIN** looked like in the tutorial
```
SELECT p.Name AS Pet_Name, o.Name as Owner_Name
FROM `bigquery-public-data.pet_records.pets` as p
INNER JOIN `bigquery-public-data.pet_records.owners` as o ON p.ID = o.Pet_ID
```

It may be useful to scroll up and review the results from when you called **head** on `posts_answers` and `posts_questions`.  

In [27]:
from time import time


answers_query = \
"""
SELECT a.id,a.body,a.owner_user_id  
FROM `bigquery-public-data.stackoverflow.posts_questions` as q
INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` as a on q.id = a.parent_id 
WHERE q.tags like '%bigquery%'
"""

answers_results = stack_overflow.query_to_pandas_safe(answers_query, max_gb_scanned=50) # query scans more than 1GB of data, but less than 2.
print(answers_results.head())
q_4.check()

         id      ...      owner_user_id
0  48732988      ...          9251751.0
1  52429303      ...         10043850.0
2  40772807      ...          1086525.0
3  42517728      ...           266531.0
4  51849078      ...           215883.0

[5 rows x 3 columns]


<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> The results don't look right. Try again.

In [28]:
# q_4.hint()
# q_4.solution()

# 5 Answer The Question
You have the merge you need. But you want a list of users who have answered many questions... which requires more work beyond your previous result.

Write a new query that selects data from the `posts_questions` and `posts_answers` tables. The results should have a single row for each user who answered at least one questions with a tag that includes the string `bigquery`. Each row in your results should have two columns:
- a column called `user_id` that contains the `owner_user_id` from the `posts_answers` table
- a column called `number_of_answers` that contains the number of answers the user has written to `bigquery` questions

In [30]:
# your code here
bigquery_experts_query = """SELECT a.owner_user_id AS user_id, COUNT(1) AS number_of_answers
                         FROM `bigquery-public-data.stackoverflow.posts_questions` AS q
                         INNER JOIN `bigquery-public-data.stackoverflow.posts_answers` AS a
                             ON q.id = a.parent_Id
                         WHERE q.tags LIKE '%bigquery%'
                         GROUP BY a.owner_user_id"""

bigquery_experts_results = stack_overflow.query_to_pandas_safe(bigquery_experts_query,max_gb_scanned=50)

print(bigquery_experts_results)
q_5.check()

         user_id  number_of_answers
0      4001094.0                 93
1      4490873.0                263
2      8689157.0                  2
3      5529445.0                 11
4      1529673.0                  5
5      4493743.0                  6
6      1623044.0                  1
7      7590690.0                  2
8      5499542.0                 43
9      7813005.0                 24
10     6356056.0                  1
11     9261416.0                  1
12     1192682.0                  1
13     3421697.0                  4
14      462923.0                  1
15      614157.0                  1
16     2570756.0                  1
17      976883.0                  2
18     2469308.0                  1
19      356708.0                  1
20     5411326.0                  1
21     4482491.0                 49
22     9564651.0                  5
23     9398998.0                 11
24     1322998.0                  1
25     4080854.0                  1
26     5660500.0            

<IPython.core.display.Javascript object>

<span style="color:#cc3333">Incorrect:</span> The results don't look right. Try again.

In [31]:
# q_5.hint()
# q_5.solution()

# Building A More Generally Useful Service

How could you convert what you've done so it's a general function a website could call on the backend to get experts on any topic?  

Think about it and then check the solution below.

In [ ]:
# q_6.solution()

# Congratulations
You know all the key components to use BigQuery and SQL effectively. Your SQL skills are sufficient to unlock many of the world's large datasets.

Want to go play with your new powers?  Kaggle has BigQuery datasets available [here](https://www.kaggle.com/datasets?sortBy=hottest&group=public&page=1&pageSize=20&size=sizeAll&filetype=fileTypeBigQuery).

# Feedback
Bring any questions or feedback to the [Learn Discussion Forum](https://www.kaggle.com/learn-forum).


---
**[SQL Micro-Course Home Page](https://www.kaggle.com/learn/SQL)**

